## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
!ls

README.md                         chronic_kidney_disease_header.txt
chronic_kidney_disease.pdf        starter-code.ipynb
chronic_kidney_disease_full.csv


In [2]:
import pandas as pd
df = pd.read_csv('chronic_kidney_disease_full.csv')

In [3]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer: 
- age
- blood pressure
- anemia
- diabetes mellititus
- sugar

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [4]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [5]:
df.shape

(400, 25)

In [6]:
drop_na_df = df.dropna()
drop_na_df.shape

(158, 25)

Answer: 158 rows would remain. This means we dropped 60% of our data. Estimates may be biased if data is not missing completely at random.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: Human error.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: Construct the model such that it tells everyone they have CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Sensitivity. We minimized false negatives

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: Everyone is told they have CKD, even if it's very unlikely they do.

### 9. Suppose that I want to construct a model where a person who does not have CKD will never be told that they do have CKD. What (very simple) model can I create that will accomplish this?

Answer: A model that tells everyone they do not have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Specificity. We minimized false positives.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: People with KCD will be told they don't have it, which means they will be less likely to address the issue.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [8]:
df.dropna(subset=['age', 'ane'], inplace=True)

In [9]:
df['has_anemia'] = df['ane'].map({'yes': 1, 'no': 0})

In [10]:
df['class'] = df['class'].map({'ckd': 1, 'notckd': 0})

In [11]:
features = ['age', 'has_anemia']
X = df[features]
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [13]:
logreg = LogisticRegression()

In [14]:
logreg.fit(X_train_sc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [58]:
logreg.score(X_train_sc, y_train)

0.7453233564938535

In [59]:
logreg.score(X_test_sc, y_test)

0.7532051282051282

In [15]:
logreg.coef_

array([[0.39136613, 1.34844657]])

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [16]:
import numpy as np

In [17]:
np.exp(0.39136613)

1.47899992053531

as age increases by 1, someone is about 1.5 times as likely to have CKD

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [18]:
np.exp(1.34844657)

3.8514379420088476

someone with anemia is about 3.9 times as likely to have CKD

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer:  
1) It's efficient.  
2) The coefficients in a logistic regression model are interpretable.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [19]:
from sklearn.metrics import confusion_matrix

In [20]:
preds = logreg.predict(X_test_sc)

In [21]:
cm = confusion_matrix(y_test, preds)

In [22]:
cm_df = pd.DataFrame(cm, columns = ['pred neg', 'pred pos'],
            index = ['actual neg', 'actual pos'])
cm_df

,pred neg,pred pos
actual neg,15,22
actual pos,6,55


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: we want to optimize for sensitivity. there can be serious consequences of someone getting a false negative result (being told they don't have CKD even though they actually do), since they will not know to address the issue.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

In [23]:
df['class'].value_counts(normalize=True)

1    0.620513
0    0.379487
Name: class, dtype: float64

Answer: with an imbalanced class, our model is more likely to ignore the smaller class. this particular CKD analysis has an imbalanced class, with about 62% of the participants having CKD.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer: with imbalanced classes, our predictions become less meaningful. it's easy to predict things correctly when when always occur that way.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 4,600 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with 5,000 observations, of which only about 150 are non-CKD individuals.

In [37]:
from sklearn.utils import resample
bootstrap = resample(df[df['class'] == 1], 
                     replace=True, 
                     n_samples=4600, 
                     random_state=42)

In [42]:
imbalanced_df = pd.concat([df, bootstrap])

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?

In [48]:
features = ['age', 'has_anemia']
X = imbalanced_df[features]
y = imbalanced_df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [49]:
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [52]:
clf = LogisticRegression()

In [53]:
clf.fit(X_train_sc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
clf.score(X_train_sc, y_train)

0.9703367183324425

In [56]:
clf.score(X_test_sc, y_test)

0.9703525641025641

In [74]:
preds = clf.predict(X_test_sc)
preds_df = pd.DataFrame(preds)
preds_df[0].value_counts()

1    1248
Name: 0, dtype: int64

The model has much higher accuracy scores with the highly imbalanced class. The model predicts that all patients are ckd-positive, and since the class is mostly ckd-positive, the model is mostly correct.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!